In [83]:
def get_input() -> list[str]:
    return get_lines_from_file('./input')

def get_test_input() -> list[str]:
    return get_lines_from_file('./test_input')

def get_lines_from_file(filepath) -> list[str]:
    with open(filepath) as f:
        return [line.strip('\n') for line in f.readlines()]

def get_str_from_file(filepath) -> str:
    with open(filepath) as f:
        return f.readline().strip('\n')

def get_int_from_file(filepath) -> int:
    with open(filepath) as f:
        return int(f.readline().strip())

def log_invocation(func):
    def logged_func(*args):
        res = func(*args)
        print(f'{func.__name__}({args}) -> {res}')
        return res
    return logged_func

In [84]:
from typing import Tuple
from collections import namedtuple
import re


INSTRUCTION_REGEX = re.compile(r'move ([\d]+) from ([\d]+) to ([\d]+)')

Instruction = namedtuple('Instruction', ['source', 'target', 'count'])

def is_crate_line(line: str) -> bool:
    return line.find('[') != -1

def parse_crate(crate: str) -> str:
    if crate[0] == '[' and crate[-1] == ']':
        return crate[1:-1]
    else:
        return ''

def parse_crate_line(line: str) -> list[str]:
    i = 0
    crates = []
    while i < len(line):
        crates.append(parse_crate(line[i:i+3]))
        i += 3
        if i < len(line):
            i += 1
    return crates

def reshape_to_columns(line_oriented: list[list[str]]) -> list[list[str]]:
    width = len(line_oriented[0])
    column_oriented = []
    for col in range(width):
        column = []
        for crate_line in line_oriented:
            column += crate_line[col]
        column_oriented.append(list(reversed(column)))
    return column_oriented

def parse_crate_stacks(input: list[str]) -> Tuple[list[list[str]], int]:
    crate_lines = []
    i = 0
    while i < len(input) and is_crate_line(input[i]):
        crate_lines.append(parse_crate_line(input[i]))
        i += 1    
    
    return reshape_to_columns(crate_lines), i

def is_instruction_line(line: str) -> bool:
    return INSTRUCTION_REGEX.match(line) != None

def parse_instruction_line(line: str) -> Instruction:
    m = INSTRUCTION_REGEX.match(line)
    return Instruction(int(m.group(2)), int(m.group(3)), int(m.group(1)))

def parse_instruction_lines(lines: list[str]) -> list[Instruction]:
    instructions = []
    i = 0
    while i < len(lines) and is_instruction_line(lines[i]):
        instructions.append(parse_instruction_line(lines[i]))
        i += 1
    return instructions

def move_crate(stacks: list[list[str]], instr: Instruction):
    for i in range(instr.count):
        stacks[instr.target-1].append(stacks[instr.source-1].pop())

def solution1(input: list[str]) -> str:
    stacks, i = parse_crate_stacks(input)

    # Skip index line and subsequent empty line
    i += 2
    
    instructions = parse_instruction_lines(input[i:])

    for instruction in instructions:
        move_crate(stacks, instruction)

    top_crates = ''
    for stack in stacks:
        top_crates += stack[-1]

    return top_crates


In [85]:
def move_crate_2(stacks: list[list[str]], instr: Instruction):
    stacks[instr.target-1] += stacks[instr.source-1][-instr.count:]
    stacks[instr.source-1] = stacks[instr.source-1][:-instr.count]

def solution2(input: list[str]) -> int:
    stacks, i = parse_crate_stacks(input)

    # Skip index line and subsequent empty line
    i += 2
    
    instructions = parse_instruction_lines(input[i:])

    for instruction in instructions:
        move_crate_2(stacks, instruction)

    top_crates = ''
    for stack in stacks:
        top_crates += stack[-1]

    return top_crates

In [86]:
solutions = [
    solution1,
    solution2,
]

test_results = [
    get_str_from_file('./test_result1'),
    get_str_from_file('./test_result2'),
]

def run_test(idx) -> bool:
    res = solutions[idx-1](get_test_input())
    test_res = test_results[idx-1]
    
    if test_res == res:
        print(f'Your solution for part {idx} works!!! :) (on the test input, that is)')
        print(f'Let`s try it on the actual input now...')
        return True
    else:
        print(f'Your solution for part {idx} does not work yet. Keep going!')
        print(f'You`ve got {res}, but the correct test result is {test_res}')
        return False

def run_solution(idx):
    sol = solutions[idx-1](get_input())
    print(f'The solution for part {idx} is: {sol}')

if run_test(1):
    run_solution(1)
    print('\nOn to part 2...\n')
    if run_test(2):
        run_solution(2)

Your solution for part 1 works!!! :) (on the test input, that is)
Let`s try it on the actual input now...
The solution for part 1 is: CWMTGHBDW

On to part 2...

Your solution for part 2 works!!! :) (on the test input, that is)
Let`s try it on the actual input now...
The solution for part 2 is: SSCGWJCRB
